Q1: Implementing an RNN for Text Generation

Task: Recurrent Neural Networks (RNNs) can generate sequences of text. You will train an LSTM-based RNN to predict the next character in a given text dataset.

1.	Load a text dataset (e.g., "Shakespeare Sonnets", "The Little Prince").
2.	Convert text into a sequence of characters (one-hot encoding or embeddings).
3.	Define an RNN model using LSTM layers to predict the next character.
4.	Train the model and generate new text by sampling characters one at a time.
5.	Explain the role of temperature scaling in text generation and its effect on randomness.


In [ ]:
import tensorflow as tf
import numpy as np
import os

# Load Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')
# Unique characters
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Vectorized text
text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Create batches of sequences
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Create input-target pairs
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                             stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS)
def generate_text(model, start_string, temperature=1.0, num_generate=500):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature  # temperature scaling
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

Length of text: 1115394 characters
Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1242s 7s/step - loss: 2.8593
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1225s 7s/step - loss: 1.8489
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1222s 7s/step - loss: 1.5940
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1180s 7s/step - loss: 1.4732
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1253s 7s/step - loss: 1.3997
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1208s 7s/step - loss: 1.3463
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1211s 7s/step - loss: 1.3069
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1212s 7s/step - loss: 1.2669
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1219s 7s/step - loss: 1.2368
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1223s 7s/step - loss: 1.2001


In [ ]:
pip install nltk

Q2: NLP Preprocessing Pipeline
Write a Python function that performs basic NLP preprocessing on a sentence. The function should do the following steps:
1.	Tokenize the sentence into individual words.
2.	Remove common English stopwords (e.g., "the", "in", "are").
3.	Apply stemming to reduce each word to its root form.
Use the sentence:
"NLP techniques are used in virtual assistants like Alexa and Siri."
The function should print:
•	A list of all tokens
•	The list after stop words are removed
•	The final list after stemming


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# First-time download of resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to fix the error

def preprocess_nlp(sentence):
    # 1. Tokenization
    tokens = word_tokenize(sentence)
    print("Tokens:", tokens)

    # 2. Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    print("After Stopword Removal:", filtered_tokens)

    # 3. Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    print("After Stemming:", stemmed_tokens)

# Example sentence
sentence = "NLP techniques are used in virtual assistants like Alexa and Siri."
preprocess_nlp(sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Tokens: ['NLP', 'techniques', 'are', 'used', 'in', 'virtual', 'assistants', 'like', 'Alexa', 'and', 'Siri', '.']
After Stopword Removal: ['NLP', 'techniques', 'used', 'virtual', 'assistants', 'like', 'Alexa', 'Siri', '.']
After Stemming: ['nlp', 'techniqu', 'use', 'virtual', 'assist', 'like', 'alexa', 'siri', '.']


In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Q3: Named Entity Recognition with SpaCy

Task: Use the spaCy library to extract named entities from a sentence. For each entity, print:
•	The entity text (e.g., "Barack Obama")
•	The entity label (e.g., PERSON, DATE)
•	The start and end character positions in the string
Use the input sentence:
"Barack Obama served as the 44th President of the United States and won the Nobel Peace Prize in 2009."


In [ ]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # stability fix
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def scaled_dot_product_attention(Q, K, V):
    # Step 1: Q . K^T
    matmul_qk = np.dot(Q, K.T)

    # Step 2: Scale by sqrt(d)
    d_k = K.shape[-1]
    scaled_scores = matmul_qk / np.sqrt(d_k)

    # Step 3: Softmax over last axis (rows)
    attention_weights = softmax(scaled_scores)

    # Step 4: Weighted sum with V
    output = np.dot(attention_weights, V)

    return attention_weights, output

# Test Inputs
Q = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
K = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
V = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

# Run attention
weights, output = scaled_dot_product_attention(Q, K, V)

# Display
print("Attention Weights:\n", weights)
print("\nOutput:\n", output)


Attention Weights:
 [[0.73105858 0.26894142]
 [0.26894142 0.73105858]]

Output:
 [[2.07576569 3.07576569 4.07576569 5.07576569]
 [3.92423431 4.92423431 5.92423431 6.92423431]]


In [ ]:
pip install transformers


Task: Implement the scaled dot-product attention mechanism. Given matrices Q (Query), K (Key), and V (Value), your function should:
•	Compute the dot product of Q and Kᵀ
•	Scale the result by dividing it by √d (where d is the key dimension)
•	Apply softmax to get attention weights
•	Multiply the weights by V to get the output
Use the following test inputs:
Q = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
K = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
V = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

Expected Output Description:
Your output should display:
1.	The attention weights matrix (after softmax)
2.	The final output matrix


In [ ]:
from transformers import pipeline

# Load pre-trained sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Input sentence
text = "Despite the high price, the performance of the new MacBook is outstanding."

# Analyze sentiment
result = sentiment_pipeline(text)[0]

# Print results
print("Label:", result['label'])
print("Confidence Score:", round(result['score'], 4))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Label: POSITIVE
Confidence Score: 0.9998
